In [1]:
import sys
import spotipy
import spotipy.util as util
import json
import requests
from furl import furl
from math import ceil
from spotipy.oauth2 import SpotifyClientCredentials

# to save some typing
import pandas as pd
import matplotlib

# to display plots in the notebook
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
cid ="3fc91dc129ff4936beb0411d23003200" 
secret = "baa0c78d715f4497a8baedd3677516db"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
artist_name = []
track_name = []
track_id = []
popularity = []

for i in range(0,10000,50):

    track_results = sp.search(q='year:2019', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        popularity.append(t['popularity'])

In [4]:
df = pd.DataFrame()

df['artist_name'] = artist_name
df['track_name'] = track_name
df['track_id'] = track_id
df['popularity'] = popularity


In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 4 columns):
artist_name    10000 non-null object
track_name     10000 non-null object
track_id       10000 non-null object
popularity     10000 non-null int64
dtypes: int64(1), object(3)
memory usage: 312.6+ KB


In [27]:
df_tracks = df.copy()
batchsize = 100

In [30]:
rows = []
None_counter = 0
for i in range(0,len(df_tracks['track_id']),batchsize):
    batch = df_tracks['track_id'][i:i+batchsize]
    feature_results = sp.audio_features(batch)
    for i, t in enumerate(feature_results):
        if t == None:
            None_counter = None_counter + 1
        else:
            rows.append(t)

In [40]:
df_tracks = pd.DataFrame(rows)
df_tracks = df_tracks.rename(columns={"id": "track_id"})

In [42]:
df_all = pd.merge(df, df_tracks, on = "track_id", how = "left", validate = "one_to_one" )

In [44]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 0 to 9999
Data columns (total 21 columns):
artist_name         10000 non-null object
track_name          10000 non-null object
track_id            10000 non-null object
popularity          10000 non-null int64
acousticness        9905 non-null float64
analysis_url        9905 non-null object
danceability        9905 non-null float64
duration_ms         9905 non-null float64
energy              9905 non-null float64
instrumentalness    9905 non-null float64
key                 9905 non-null float64
liveness            9905 non-null float64
loudness            9905 non-null float64
mode                9905 non-null float64
speechiness         9905 non-null float64
tempo               9905 non-null float64
time_signature      9905 non-null float64
track_href          9905 non-null object
type                9905 non-null object
uri                 9905 non-null object
valence             9905 non-null float64
dtypes: float64